In [ ]:
# 📦 Install dependencies (if running on Colab or fresh environment)
!pip install -q transformers datasets librosa jiwer


In [ ]:
import torch

if torch.cuda.is_available():
    print("✅ GPU détecté :", torch.cuda.get_device_name(0))
else:
    raise SystemError("🚨 GPU non disponible. Va dans le menu Exécution > Modifier le type d'exécution > GPU.")


✅ GPU détecté : Tesla T4


In [ ]:
# ✅ Load Whisper model and processor
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch

model_checkpoint = "openai/whisper-small"
processor = WhisperProcessor.from_pretrained(model_checkpoint)
model = WhisperForConditionalGeneration.from_pretrained(model_checkpoint)
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="fr", task="transcribe")
model.config.suppress_tokens = []
model.eval()


WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (f

In [ ]:
!pip install --upgrade "fsspec<=2023.6.0"
!pip install -q datasets transformers accelerate torchaudio jiwer evaluate

from huggingface_hub import login
login(token="")

# 🔄 Load and clean dataset (Darija in Latin script)
from datasets import load_dataset

# Replace with your actual dataset
ds = load_dataset("atlasia/DODa-audio-dataset", download_mode="force_redownload")  # e.g., "yourname/darja-dataset"

def is_valid(example):
    txt = example.get("darija_Arab_new")
    return (
        txt is not None and txt.strip() != "" and
        example.get("audio", {}).get("array") is not None and
        sum(example["audio"]["array"]) != 0
    )

ds_cleaned = ds.filter(is_valid)



Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/12743 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


Filter:   0%|          | 0/12743 [00:00<?, ? examples/s]

In [ ]:
# 🎯 Prepare a single sample for inference
import torch

# ⚠️ Spécifier le split train
example = ds_cleaned["train"][0]

# Préparation de l'entrée audio
input_features = processor(example["audio"]["array"], sampling_rate=16000, return_tensors="pt").input_features

# Génération avec le modèle
generated_ids = model.generate(
    input_features.to(model.device),
    max_length=128,
    no_repeat_ngram_size=3,
    forced_decoder_ids=processor.get_decoder_prompt_ids(language="ar", task="transcribe")
)

# Transcription
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

# Affichage
print("🔊 Audio transcription:", transcription)
print("📝 Référence :", example["darija_Arab_new"])


Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


🔊 Audio transcription:  هم مخبنشي حاجة أنا متقن
📝 Référence : هوما مخبيين شي حاجة انا متيقن


In [ ]:
# 5. Fonction de prétraitement
def prepare_dataset(example):
    audio = example["audio"]
    example["input_features"] = processor.feature_extractor(
        audio["array"], sampling_rate=audio["sampling_rate"]
    ).input_features[0]
    example["labels"] = processor.tokenizer(
        example["darija_Arab_new"],
        truncation=True,
        max_length=225,
        return_tensors=None
    ).input_ids
    return example

# 6. Prétraiter chaque split
ds_preprocessed = {
    split: ds_cleaned[split].map(
        prepare_dataset,
        remove_columns=ds_cleaned[split].column_names
    )
    for split in ds_cleaned
}

# 7. DataCollator personnalisé
class WhisperDataCollator:
    def __init__(self, processor):
        self.processor = processor

    def __call__(self, features):
        input_features = [{"input_features": f["input_features"]} for f in features]
        labels = [{"input_ids": f["labels"]} for f in features]

        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        label_batch = self.processor.tokenizer.pad(labels, return_tensors="pt")
        batch["labels"] = label_batch["input_ids"].masked_fill(label_batch.attention_mask.ne(1), -100)
        return batch

# 8. Initialiser le DataCollator
data_collator = WhisperDataCollator(processor)


Map:   0%|          | 0/12720 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, Seq2SeqTrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
from transformers import WhisperForConditionalGeneration
import types

# Charger modèle et config LoRA
base_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["k_proj", "v_proj"]
)
model = get_peft_model(base_model, peft_config)

# Patcher base_model.forward pour supprimer input_ids et inputs_embeds
base_forward = model.base_model.forward

def base_model_forward_patch(self, *args, **kwargs):
    for arg in ["input_ids", "inputs_embeds"]:
        if arg in kwargs:
            kwargs.pop(arg)
    return base_forward(*args, **kwargs)

model.base_model.forward = types.MethodType(base_model_forward_patch, model.base_model)
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-darja-lora",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=1e-4,
    warmup_steps=100,
    num_train_epochs=2,     # <--- 2 epochs here
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    report_to="none",
    fp16=False,            # fp16 usually unsupported on CPU
    predict_with_generate=True,
    generation_max_length=225,
    remove_unused_columns=False,
    dataloader_pin_memory=False,
)


# Créer Trainer (avec ton ds_preprocessed et data_collator définis)
# Créer Trainer sans eval_dataset
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_preprocessed["train"],  # ✅ nécessaire
    data_collator=data_collator,             # ✅ indispensable
)


# Lancer entraînement
trainer.train()


No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
100,3.599800
200,1.528900
300,0.736000
400,0.613200
500,0.598100
600,0.588600
700,0.527100
800,0.523100
900,0.543700
1000,0.524100


TrainOutput(global_step=3180, training_loss=0.595494270324707, metrics={'train_runtime': 12883.4752, 'train_samples_per_second': 1.975, 'train_steps_per_second': 0.247, 'total_flos': 7.3740236488704e+18, 'train_loss': 0.595494270324707, 'epoch': 2.0})

In [ ]:
# Dossier de sortie
output_dir = "./whisper-darja-lora-final"

# 1. Sauvegarder le modèle LoRA
model.save_pretrained(output_dir)

# 2. Sauvegarder le processor (tokenizer + feature extractor)
processor.save_pretrained(output_dir)

print(f"✅ Modèle et processor sauvegardés dans : {output_dir}")


✅ Modèle et processor sauvegardés dans : ./whisper-darja-lora-final


In [ ]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor
from peft import PeftModel, PeftConfig

# 1. Charger le processor
processor = WhisperProcessor.from_pretrained("./whisper-darja-lora-final")

# 2. Charger le modèle de base
base_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

# 3. Appliquer les poids LoRA sur le modèle de base
model = PeftModel.from_pretrained(base_model, "./whisper-darja-lora-final")

# 4. Passer en eval mode si besoin
model.eval()


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): WhisperForConditionalGeneration(
      (model): WhisperModel(
        (encoder): WhisperEncoder(
          (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
          (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
          (embed_positions): Embedding(1500, 768)
          (layers): ModuleList(
            (0-11): 12 x WhisperEncoderLayer(
              (self_attn): WhisperAttention(
                (k_proj): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=False)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=8, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=8, out_features=768, 

In [ ]:
from datasets import load_dataset

ds = load_dataset("Snousnou/Moroccan-Darija-ASR")
example = ds["train"][2]  # index 2 pour “bghit nkhles dariba”

print("Audio durée:", example["file"], "s")
print("Transcription:", example["text"])


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/2 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/6 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1 [00:00<?, ? examples/s]

KeyError: 'audioduration'

In [ ]:
from datasets import load_dataset

ds = load_dataset("Snousnou/Moroccan-Darija-ASR")
example = ds["train"][2]
print(example.keys())


dict_keys(['file', 'text'])


In [ ]:
from datasets import load_dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import PeftModel
import torch

# Charger dataset
ds = load_dataset("Snousnou/Moroccan-Darija-ASR")
example = ds["train"][2]

print("Clés disponibles :", example.keys())
print("Référence :", example["text"])

# Récupérer audio (array + sample_rate)
audio_array = example["file"]["array"]
sample_rate = example["file"]["sampling_rate"]

# Charger modèle
processor = WhisperProcessor.from_pretrained("./whisper-darja-lora-final")
base_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model = PeftModel.from_pretrained(base_model, "./whisper-darja-lora-final")
model.eval()

# Préparer features
inputs = processor(audio_array, sampling_rate=sample_rate, return_tensors="pt").input_features.to(model.device)

# Générer transcription
with torch.no_grad():
   generated_ids = model.base_model.generate(
    inputs,
    max_length=128,
    forced_decoder_ids=processor.get_decoder_prompt_ids(language="ar", task="transcribe")
)


transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print("Prédiction :", transcription)


Clés disponibles : dict_keys(['file', 'text'])
Référence : bghit nkhles dariba
Prédiction : تغيط نخلص الطريبة


In [ ]:
from datasets import load_dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch

# Charger dataset et un exemple
ds = load_dataset("Snousnou/Moroccan-Darija-ASR")
example = ds["train"][2]
audio_array = example["file"]["array"]
sample_rate = example["file"]["sampling_rate"]

# Charger processor et modèle de base Whisper-small
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
base_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
base_model.eval()

# Préparer les features
inputs = processor(audio_array, sampling_rate=sample_rate, return_tensors="pt").input_features

# Générer transcription avec Whisper-small (base)
with torch.no_grad():
    generated_ids_base = base_model.generate(
        inputs,
        max_length=128,
        forced_decoder_ids=processor.get_decoder_prompt_ids(language="ar", task="transcribe")
    )
transcription_base = processor.batch_decode(generated_ids_base, skip_special_tokens=True)[0]

print("📝 Référence :", example["text"])
print("🤖 Whisper-small base :", transcription_base)


📝 Référence : bghit nkhles dariba
🤖 Whisper-small base :  بريبن خلص بطاريب


In [ ]:
from datasets import load_dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import PeftModel
import torch
from jiwer import wer

# Charger dataset (tu peux changer 'train' en 'test' si tu préfères)
ds = load_dataset("Snousnou/Moroccan-Darija-ASR", split="train")

# Charger processor et modèles
processor = WhisperProcessor.from_pretrained("./whisper-darja-lora-final")
base_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
base_model.eval()

base_model.to("cuda" if torch.cuda.is_available() else "cpu")

# Charger modèle LoRA
base_model_for_lora = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model_lora = PeftModel.from_pretrained(base_model_for_lora, "./whisper-darja-lora-final")
model_lora.eval()
model_lora.to("cuda" if torch.cuda.is_available() else "cpu")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Limite le nombre d'exemples pour test rapide
max_examples = 10

wers_base = []
wers_lora = []

for i, example in enumerate(ds):
    if i >= max_examples:
        break

    audio = example["file"]["array"]
    sample_rate = example["file"]["sampling_rate"]
    reference = example["text"]

    inputs = processor(audio, sampling_rate=sample_rate, return_tensors="pt").input_features.to(device)

    # Base model generate
    with torch.no_grad():
        generated_ids_base = base_model.generate(
            inputs,
            max_length=128,
            forced_decoder_ids=processor.get_decoder_prompt_ids(language="ar", task="transcribe")
        )
    pred_base = processor.batch_decode(generated_ids_base, skip_special_tokens=True)[0]

    # LoRA model generate (attention à generate sur base_model)
    with torch.no_grad():
        generated_ids_lora = model_lora.base_model.generate(
            inputs,
            max_length=128,
            forced_decoder_ids=processor.get_decoder_prompt_ids(language="ar", task="transcribe")
        )
    pred_lora = processor.batch_decode(generated_ids_lora, skip_special_tokens=True)[0]

    # Calcul WER (tu peux normaliser les textes si besoin)
    wer_base = wer(reference, pred_base)
    wer_lora = wer(reference, pred_lora)

    wers_base.append(wer_base)
    wers_lora.append(wer_lora)

    print(f"\nExemple {i+1}:")
    print("Référence     :", reference)
    print("Prédiction Base :", pred_base)
    print(f"WER Base      : {wer_base:.3f}")
    print("Prédiction LoRA :", pred_lora)
    print(f"WER LoRA      : {wer_lora:.3f}")

print("\n=== Résultats moyens ===")
print(f"WER moyen Base : {sum(wers_base)/len(wers_base):.3f}")
print(f"WER moyen LoRA : {sum(wers_lora)/len(wers_lora):.3f}")



Exemple 1:
Référence     : bghit nsift lflous
Prédiction Base :  ونحن نقوم بعمل سفت الفلوس
WER Base      : 1.667
Prédiction LoRA : بغيت نسافط الفلوس
WER LoRA      : 1.000

Exemple 2:
Référence     : ch7al f solde
Prédiction Base :  شهل في السول
WER Base      : 1.000
Prédiction LoRA : شحال فالسول
WER LoRA      : 1.000

Exemple 3:
Référence     : bghit nkhles dariba
Prédiction Base :  بريبن خلص بطاريب
WER Base      : 1.000
Prédiction LoRA : تغيط نخلص الطريبة
WER LoRA      : 1.000

Exemple 4:
Référence     : ch7al flcompte
Prédiction Base :  شهل عندي في الكنت
WER Base      : 2.000
Prédiction LoRA : شحال عندي في الكنت
WER LoRA      : 2.000

Exemple 5:
Référence     : bghit nkhrej lflous
Prédiction Base :  ريتم خرزر فلو
WER Base      : 1.000
Prédiction LoRA : غيط نخرج نشفو
WER LoRA      : 1.000

Exemple 6:
Référence     : bghit nkhles lma o do
Prédiction Base :  بريد نخلص
WER Base      : 1.000
Prédiction LoRA : بغيت نخلص
WER LoRA      : 1.000

=== Résultats moyens ===
WER moyen Base : 1.27

In [ ]:
def simple_arabic_to_latin(text):
    mapping = {
        'ب': 'b', 'ر': 'r', 'ي': 'y', 'د': 'd',
        'ن': 'n', 'خ': 'kh', 'ل': 'l', 'ص': 's',
        'ط': 't', 'ا': 'a', 'ح': 'h', 'ق': 'q',
        'س': 's', 'م': 'm', 'و': 'w', 'ف': 'f',
        'ش': 'sh', 'ع': 'a', 'غ': 'gh', 'ك': 'k',
        'ت': 't', 'ج': 'j', 'ذ': 'dh', 'ز': 'z',
        'ض': 'd', 'ظ': 'z', 'ه': 'h', 'ي': 'y',
        'ء': "'", 'ؤ': 'w', 'ئ': 'y',
        # ajoute les autres lettres si besoin
    }
    latin = ""
    for ch in text:
        latin += mapping.get(ch, ch)  # si lettre inconnue, la garder brute
    return latin

# Exemple d’utilisation
ar_text = "بريد نخلص"
print(simple_arabic_to_latin(ar_text))  # devrait afficher : brid nkhls


In [ ]:
import os

print("Contenu de ./whisper-darja-lora :")
print(os.listdir("./whisper-darja-lora"))


Contenu de ./whisper-darja-lora :
['checkpoint-3180', 'checkpoint-2000', 'checkpoint-1000', 'checkpoint-1500', 'checkpoint-2500', 'checkpoint-3000', 'checkpoint-500']


In [ ]:


import shutil

shutil.make_archive("whisper-darja-lora-final", 'zip', output_dir)
print("Archive whisper-darja-lora.zip créée.")


Archive whisper-darja-lora.zip créée.


In [ ]:
from datasets import load_dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import PeftModel
import torch
from jiwer import wer
import types

ds = load_dataset("atlasia/DODa-audio-dataset", split="train")

processor = WhisperProcessor.from_pretrained("./whisper-darja-lora-final")

base_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
base_model.eval()
base_model.to("cuda" if torch.cuda.is_available() else "cpu")

base_model_for_lora = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model_lora = PeftModel.from_pretrained(base_model_for_lora, "./whisper-darja-lora-final")
model_lora.eval()
model_lora.to("cuda" if torch.cuda.is_available() else "cpu")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Patch nécessaire sur forward
base_forward = model_lora.base_model.forward
def base_model_forward_patch(self, *args, **kwargs):
    for arg in ["input_ids", "inputs_embeds"]:
        if arg in kwargs:
            kwargs.pop(arg)
    return base_forward(*args, **kwargs)
model_lora.base_model.forward = types.MethodType(base_model_forward_patch, model_lora.base_model)

max_examples = 10

wers_base = []
wers_lora = []

for i, example in enumerate(ds):
    if i >= max_examples:
        break

    audio = example["audio"]["array"]
    sample_rate = example["audio"]["sampling_rate"]
    reference = example["darija_Arab_new"]

    inputs = processor(audio, sampling_rate=sample_rate, return_tensors="pt").input_features.to(device)

    with torch.no_grad():
        generated_ids_base = base_model.generate(
            inputs,
            max_length=128,
            forced_decoder_ids=processor.get_decoder_prompt_ids(language="ar", task="transcribe")
        )
    pred_base = processor.batch_decode(generated_ids_base, skip_special_tokens=True)[0]

    with torch.no_grad():
        generated_ids_lora = model_lora.base_model.generate(
            input_features=inputs,
            max_length=128,
            forced_decoder_ids=processor.get_decoder_prompt_ids(language="ar", task="transcribe")
        )
    pred_lora = processor.batch_decode(generated_ids_lora, skip_special_tokens=True)[0]

    wer_base = wer(reference, pred_base)
    wer_lora = wer(reference, pred_lora)

    wers_base.append(wer_base)
    wers_lora.append(wer_lora)

    print(f"\nExemple {i+1}:")
    print("Référence     :", reference)
    print("Prédiction Base :", pred_base)
    print(f"WER Base      : {wer_base:.3f}")
    print("Prédiction LoRA :", pred_lora)
    print(f"WER LoRA      : {wer_lora:.3f}")

print("\n=== Résultats moyens ===")
print(f"WER moyen Base : {sum(wers_base)/len(wers_base):.3f}")
print(f"WER moyen LoRA : {sum(wers_lora)/len(wers_lora):.3f}")



Exemple 1:
Référence     : هوما مخبيين شي حاجة انا متيقن
Prédiction Base :  هم مخبنشي حاجة أنا متقن
WER Base      : 0.833
Prédiction LoRA : هما مخبين شي حاجة انا متقن
WER LoRA      : 0.500

Exemple 2:
Référence     : باينة هوما كيحاولو يبقاو مبردين
Prédiction Base :  بينهما كحولوا بقام بردين
WER Base      : 1.000
Prédiction LoRA : بعينة هما كيحاولو يبقاهم بردين
WER LoRA      : 0.800

Exemple 3:
Référence     : لوطيلات مبيناش فيهم مريحين بزاف
Prédiction Base :  لو طيلات ما بيناش فيهم مورحين بالزيف
WER Base      : 1.200
Prédiction LoRA : الوطيلات مبيناش فيهم مريحين بزاف
WER LoRA      : 0.200

Exemple 4:
Référence     : غالبا غيجريو عليه من الخدمة
Prédiction Base :  غالباً غيروا عليهم الخدمة
WER Base      : 0.800
Prédiction LoRA : غالبا نغيجريو عليه من الخدمة
WER LoRA      : 0.200

Exemple 5:
Référence     : طبعا راه مكتئب
Prédiction Base :  طبعا لنرحمك تائب
WER Base      : 0.667
Prédiction LoRA : طبعا راه مكتاب
WER LoRA      : 0.333

Exemple 6:
Référence     : كيبان ليا غنمشي
Prédiction